In [5]:
import torch
import torch.optim as optim
from torchvision import transforms
from torchvision import datasets
import torch.nn as nn
import ssl
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from matplotlib import pyplot as plt
import math
import time
import os
import torch.nn.functional as F

ssl._create_default_https_context = ssl._create_unverified_context

EPOCH = 10
epochs = EPOCH
pre_epoch = 0

def PLOTIMAGE(acc,val_acc,loss,val_loss):
    # 绘制精度曲线
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.legend(('Training accuracy', 'validation accuracy'))
    plt.figure()
    # 绘制损失曲线
    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.legend(('Training loss', 'validation loss'))
    plt.title('Training and validation loss')
    plt.show()
    return 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model.fc = nn.Linear(in_features=2048, out_features=2, bias=True)
base_dir = './cat_and_dog'
#指定每一种数据的位置
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
BATCH_SIZE = 16
LR = 0.0001
net = model.to(device)
# define loss funtion & optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9,weight_decay=0.1)
optimizer = optim.Adam(net.parameters(), lr=LR)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
traindata_dirname = train_dir
train_data = datasets.ImageFolder(traindata_dirname, preprocess)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

testdata_dirname = validation_dir
test_data = datasets.ImageFolder(testdata_dirname, preprocess)
testloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE * 2, shuffle=False)
if __name__ == "__main__":
    Test_acc = 0
    Test_AUC = 0
    Test_Sen = 0
    Test_Spe = 0
    Test_npv = 0
    Test_ppv = 0
    for epoch in range(pre_epoch, EPOCH):
        since = time.time()
        print('\nEpoch: %d' % (epoch + 1))
        net.train()
        sum_loss = 0.0
        correct = 0.0
        total = 0.0
        train_acc = 0
        label_train = []
        prob_train = []
        for i, data in enumerate(trainloader, 0):
            length = len(trainloader)
            # print("length",length)
            inputs, label = data
            inputs, label = inputs.to(device), label.to(device)
            optimizer.zero_grad()
            # forward & backward
            outputs = net(inputs)

            # print("outputs:",outputs)
            print("Before softmax shape: ", outputs.shape)
            probabilities = torch.nn.functional.softmax(outputs, dim=0)
            print("After softmax shape: ", probabilities.shape)
            loss = criterion(probabilities, label)
            loss.backward()
            optimizer.step()
            # print ac & loss in each batch
            sum_loss += loss.item()
            # print("probabilities:",probabilities)
            _, predicted = torch.max(probabilities, 1)
            prob_train.extend(predicted.cpu().data.numpy())
            label_train.extend(label.cpu().numpy())
            # print("predicted",predicted)
            # print("label",label)
            total += label.size(0)
            correct += (predicted == label).sum()
            # print("size",label.size(0))
            # print("correct",correct)
            # print("total",total)
            train_acc = correct / total
            print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
                  % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * train_acc))
        TN, FP, FN, TP = confusion_matrix(label_train, prob_train).ravel()
        Sen = float(TP) / (TP + FN)
        Spe = float(TN) / (FP + TN)
        npv = float(TN) / (TN + FN)
        ppv = float(TP) / (TP + FP)
        time_elapsed = time.time() - since
        print('Train\'s ac is: %.3f%%' % (100 * train_acc))
        print('Training complete in {}s'.format(time_elapsed))
        print('Waiting Test...')
        prob_test = []
        label_test = []
        with torch.no_grad():
            correct_test = 0
            total_test = 0
            since = time.time()
            for data in testloader:
                net.eval()
                images, labels_test = data
                images, labels_test = images.to(device), labels_test.to(device)
                outputs_test = net(images)
                probabilities_test = torch.nn.functional.softmax(outputs_test, dim=1)
                loss = criterion(probabilities_test, labels_test)
                sum_test = probabilities_test[:, 0] + probabilities_test[:, 1]
                _, predicted_test = torch.max(probabilities_test, 1)  # 每行最大值
                prob_test.extend(predicted_test.data.cpu().numpy())
                label_test.extend(labels_test.data.cpu().numpy())
                total_test += labels_test.size(0)
                correct_test += (predicted_test == labels_test).sum()
            test_acc = correct_test / total_test
            time_elapsed = time.time() - since
            print('Test\'s ac is: %.3f%%' % (100 * test_acc))
            print('Testing complete in {}s'.format(time_elapsed))
            # print(label_all,prob_all)
            TN_tt, FP_tt, FN_tt, TP_tt = confusion_matrix(label_test, prob_test).ravel()
            Sen_tt = float(TP_tt) / (TP_tt + FN_tt)
            Spe_tt = float(TN_tt) / (FP_tt + TN_tt)
            npv_tt = float(TN_tt) / (TN_tt + FN_tt)
            ppv_tt = float(TP_tt) / (TP_tt + FP_tt)
            num = 0


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch: 1
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Size([16, 2])
[epoch:1, iter:1] Loss: 0.691 | Acc: 56.250% 
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Size([16, 2])
[epoch:1, iter:2] Loss: 0.678 | Acc: 78.125% 
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Size([16, 2])
[epoch:1, iter:3] Loss: 0.671 | Acc: 85.417% 
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Size([16, 2])
[epoch:1, iter:4] Loss: 0.664 | Acc: 89.062% 
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Size([16, 2])
[epoch:1, iter:5] Loss: 0.659 | Acc: 91.250% 
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Size([16, 2])
[epoch:1, iter:6] Loss: 0.655 | Acc: 92.708% 
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Size([16, 2])
[epoch:1, iter:7] Loss: 0.652 | Acc: 93.750% 
Before softmax shape:  torch.Size([16, 2])
After softmax shape:  torch.Si

KeyboardInterrupt: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
path = '/content/drive/MyDrive/'
os.chdir(path)

Mounted at /content/drive
